# Advent of Code 2023

generic import

In [ ]:
%load_ext autoreload
%autoreload 2

import logging
import importlib.util
from pathlib import Path
import polars as pl

from logging import DEBUG, INFO
VERBOSE = 5

parent_dir = Path().resolve()

# configure root logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)
consoleHandler = logging.StreamHandler()
formatter = logging.Formatter('%(message)s')
consoleHandler.setFormatter(formatter)
logger.addHandler(consoleHandler)

def set_log_level(log_level=INFO):
    logger = logging.getLogger()
    logger.setLevel(log_level)

def get_input_path(day, test=None, parent_dir=parent_dir):
    day_str = str(day).zfill(2)
    input_path = parent_dir.joinpath(f'day{day_str}/koen/data/input.txt')
    log_level = INFO
    if test:
        input_path = input_path.with_name(input_path.stem + f'_test{test}.txt')
        log_level = DEBUG
    set_log_level(log_level)
    return input_path

def path_import(absolute_path):
    spec = importlib.util.spec_from_file_location(absolute_path.stem, absolute_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

def dynamic_load(day):
    day_str = str(day).zfill(2)
    module_path = parent_dir.joinpath(f'day{day_str}/koen/day{day_str}.py')
    return path_import(module_path)
    

## Day 01: Trebuchet?!

sql extravaganza in duckdb using the [DBeaver extension](https://duckdb.org/docs/archive/0.9.2/guides/sql_editors/dbeaver).

## Day 02: Cub Conundrum

crawling back to python

In [ ]:
day = 2
day2 = dynamic_load(day)
input_path = get_input_path(day=day, test=None)

### part 1

In [ ]:
max_red = 12
max_green = 13
max_blue = 14
df = day2.parse_input(input_path)
df

In [ ]:
df = (
    df
    .with_columns(
        pl.col('blue').list.max().alias('blue_max'),
        pl.col('red').list.max().alias('red_max'),
        pl.col('green').list.max().alias('green_max')
    )
)
df

In [ ]:
(
    df
    .filter(
        pl.col('blue_max').le(max_blue)
        & pl.col('red_max').le(max_red)
        & pl.col('green_max').le(max_green)
    )
    .select(
        pl.col('game').sum()
    )
)

### part 2

In [ ]:
(
    df
    .select(
        (
            pl.col('blue_max') * pl.col('red_max') * pl.col('green_max')
        ).sum()
    )
)